# Import Libary

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import shutil
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, MobileNetV2, ResNet50
from google.colab import files

In [ ]:
!pip install opendatasets
import opendatasets as od

# Load Dataset

In [ ]:
#donwload dataset menggunakan url kaggle
# needs: kaggle API (goto: 'https://www.kaggle.com/me/account')
od.download("https://www.kaggle.com/datasets/achmadnoer/alfabet-bisindo")

In [ ]:
data_dir = "alfabet-bisindo/Citra BISINDO"

# Memuat dataset
batch_size = 32
img_height = 224
img_width = 224

# Membuat objek ImageDataGenerator untuk training dan validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.2, 1.0],
    channel_shift_range=0.2,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    subset='training'
)

validation_generator = val_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation'
)

class_names = list(train_generator.class_indices.keys())
print(class_names)

# Visualisasi dataset
plt.figure(figsize=(10, 10))
for images, labels in train_generator:
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(list(train_generator.class_indices.keys())[int(labels[i])])
        plt.axis("off")
    break
plt.show()

# Model

In [ ]:
# return len class
num_classes = len(class_names)

## model 1

In [ ]:
model = Sequential([

    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)), #input RGB
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

## model 2

In [ ]:
model = Sequential([

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),  # input RGB
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')

])

## Model 3 dengan menggunakan VGG16

In [ ]:
base_model = VGG16(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')

# trainable beberapa lapisan terakhir dari VGG16 untuk fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-4]:  # frezee semua lapisan kecuali 4 lapisan terakhir
    layer.trainable = False

model = Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(1072, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])


## Model 4 dengan menggunakan MobileNetV2

In [ ]:
# load model MobileNetV2 tanpa lapisan top (fully connected layers)
base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = True  # false: freeze layer base model

model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

## Model 5 dengan menggunakan ResNet50

In [ ]:
base_model = ResNet50(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

## Compile Model

In [ ]:
# compile model
model.compile(
    optimizer= tf.keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# fungsi callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks= [early_stopping, lr_scheduler]
)


In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation accuracy: {accuracy}")

# Convert model ke TensorFlow js dan TensorFlow Lite

In [ ]:
#save model
model.save('model')

## convert ke TensorFlow.js

In [ ]:
pip install tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model model model_BISINDO

In [ ]:
# Path ke direktori model TensorFlow.js
tfjs_model_path = 'model_BISINDO'

# Path untuk menyimpan file zip
zip_file_path = 'model_BISINDO.zip'

# Mengarsipkan direktori model ke file zip
shutil.make_archive(base_name=zip_file_path.replace('.zip', ''), format='zip', root_dir=tfjs_model_path)

print(f'Model has been zipped and saved to {zip_file_path}')

In [ ]:
files.download(zip_file_path)

## Convert ke TensorFlow Lite

In [ ]:
# convert model ke TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model('model')
tflite_model = converter.convert()

# save model ke file .tflite
with open('ml_model_BISINDO.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
files.download('ml_model_BISINDO.tflite')